# GameDay2020 Notebook

The standard dilemma of fantasy baseball is who to choose next: the best available player but in a position you've already filled?; the best at a rare position?; the next best closer because they're going fast?  Whatever choice you make will have consequences, as the next player will no longer have your pick as a choice, and so on.  Ideally you would forecast how each of those would play out and take the best option --- that's exactly what this code does: it identifies the best player left in each of the positions you still need to fill, simulates the rest of draft in each of those scenarios, and returns the best choice.

And it's easy to use!  After [cloning](https://github.com/wrapgenius/GameDay2020) GameDay2020 (and provided you have Jupyter Notebooks with Python 3 installed, and a spreadsheet program) you should have everything you need to do a live roto draft.

The code is based on three ingredients: *projections*, *rankings*, and *injuries*:  
- Projections are estimates of player performances for 2020, and are based on data scraped from [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips) (model = ZiPS, Steamer, or TheBat.) 
- Rankings are draft-order recommendations.  The default (i.e.; ranking_method = 'Yahoo') is to use Yahoo Rankings (from Feb 22, 2020).  Others include [ESPN](https://www.espn.com/fantasy/baseball/story/_/id/28285423/fantasy-baseball-2020-rankings-roto-rotisserie), [FantasyPros](https://www.fantasypros.com/mlb/rankings/overall.php), and [Rotographs](https://fantasy.fangraphs.com), or you can put in your own.  Yahoo for yahoo leagues makes most sense since it most closely imitates autodraft.    
- Injured players to be removed from draft consideration are located in the Injuries2020.xlsx file found in the Injured_List_Spreadsheets directory. 

_Your_ roto league may have more or less than 12 teams, or use different stats; that ok!  Declare them when defining the Draft object: most of the obvious stats are included, more obscure ones may require hacking into the [fangraphs_projection_2020](https://github.com/wrapgenius/GameDay2020/blob/master/GameDayFunctions/fangraphs_projection_2020.py).  

Note, if you choose to shuffle_picks = True, it will not default to the next best pick, but draw a random number between 0 and 1 and compare it to the sigmoid function 1/1+exp(-(pick_number - avg_pick_number)).  So say it's the 15th pick and Trevor Story, who is typically drafted 10th is available, there is an 85% chance you'll select him.  Also note, it can't _yet_ do AL or NL only. 

Below are examples of the different ways the code can be run.  

In [1]:
# Load packages.  Must have pandas and numpy.  

import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import copy
from GameDayFunctions.fangraphs_projection_2020 import Projection
from GameDayFunctions.draft_2020 import Draft

In [2]:
year = 2020
path_data = "projections/"
path_drafts = "Draft_Pick_Spreadsheets/"
shuffle_picks = False

First need a **Projection** object.  Projections are scraped from [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips), and include ZiPS, Steamer, and TheBat.  
In this example I create a dictionary of the three projections.  

In [3]:
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS',ranking_method = 'Yahoo')
player_ranking_steam = Projection(path_data=path_data,year=year,model='Steamer',ranking_method = 'FantasyPros')
player_ranking_thebat = Projection(path_data=path_data,year=year,model='TheBat',ranking_method = 'ESPN')
player_rankings = {"ZiPS":player_ranking_zips, "Steamer":player_ranking_steam, "TheBat":player_ranking_thebat}

Next need a **Draft** object.  The default, e.g.; Draft(player_rankings['ZiPS']), uses 12 teams and the roster spots/roto stats shown in this example. 

In [4]:
# Enter your roto-league-specific stats.  The default settings are shown here.  
draft_position = 2
number_teams = 12
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5}
batter_stats  = ['AB','R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS']
pitcher_stats = ['IP','W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV'] 

# Choose the projection system you prefer ()
player_projections = player_rankings['ZiPS']

# Get an instance of the Draft Class with your league-specific details and projection preference.
zips_example = Draft(player_projections, 
                     draft_position = draft_position, 
                     number_teams = number_teams,
                     roster_spots = roster_spots,
                     batter_stats = batter_stats,
                     pitcher_stats = pitcher_stats)

To use while drafting in real-time, simply record the draft results in a spreadsheet (.xlsx, so if you're using Numbers, need to export to Excel) and run the cell. 

Examples below show results for after 2, 3, and 4 rounds.  Notice that at each stage the projected placement in the standings improves.  

In [5]:
# Example pick in the 2nd round

zipsT = Draft(player_rankings['ZiPS'], draft_position = 5)
player_to_pick, projected_roto_stats = zipsT.draft_from_list_and_find_best_pick(draft_pick_file = 'TestPicksround2.xlsx', path_list = path_drafts)

Team 1 picking Mike Trout for OF
Team 2 picking Mookie Betts for OF
Team 3 picking Alex Bregman for SS
Team 4 picking Anthony Rendon for 3B
Team 5 picking Nolan Arenado for 3B
Team 6 picking Jacob deGrom for SP
Team 7 picking Christian Yelich for OF
Team 8 picking Juan Soto for OF
Team 9 picking Cody Bellinger for OF
Team 10 picking Ronald Acuna for OF
Team 11 picking Justin Verlander for SP
Team 12 picking Francisco Lindor for SS
Team 12 picking Jose Altuve for 2B
Team 11 picking Max Scherzer for SP
Team 10 picking Fernando Tatis for SS
Team 9 picking Trevor Story for SS
Team 8 picking Charlie Blackmon for OF
Team 7 picking Whit Merrifield for 2B
Team 6 picking Freddie Freeman for 1B
Finding Best Pick For Team 5
Stored Result for Pick 0 [21/12] J.D. Martinez OF whose placement/score is [11, 123.0]
Stored Result for Pick 1 [21/21] Xander Bogaerts SS whose placement/score is [11, 114.0]
Stored Result for Pick 2 [21/22] Jack Flaherty SP whose placement/score is [8, 158.5]
Stored Result f

In [6]:
# Example pick in the 3rd round

zipsT = Draft(player_rankings['ZiPS'], draft_position = 5)
player_to_pick, projected_roto_stats = zipsT.draft_from_list_and_find_best_pick(draft_pick_file = 'TestPicksround3.xlsx', path_list = path_drafts)

Team 1 picking Mike Trout for OF
Team 2 picking Mookie Betts for OF
Team 3 picking Alex Bregman for SS
Team 4 picking Anthony Rendon for 3B
Team 5 picking Nolan Arenado for 3B
Team 6 picking Jacob deGrom for SP
Team 7 picking Christian Yelich for OF
Team 8 picking Juan Soto for OF
Team 9 picking Cody Bellinger for OF
Team 10 picking Ronald Acuna for OF
Team 11 picking Justin Verlander for SP
Team 12 picking Francisco Lindor for SS
Team 12 picking Jose Altuve for 2B
Team 11 picking Max Scherzer for SP
Team 10 picking Fernando Tatis for SS
Team 9 picking Trevor Story for SS
Team 8 picking Charlie Blackmon for OF
Team 7 picking Whit Merrifield for 2B
Team 6 picking Freddie Freeman for 1B
Team 5 picking Trea Turner for SS
Team 4 picking Bryce Harper for OF
Team 3 picking Chris Sale for SP
Team 2 picking Walker Buehler for SP
Team 1 picking Shane Bieber for SP
Team 1 picking Stephen Strasburg for SP
Team 2 picking Josh Hader for RP
Team 3 picking Rafael Devers for 3B
Team 4 picking Ketel Ma

In [7]:
# Example pick in the 4th round

zipsT = Draft(player_rankings['ZiPS'], draft_position = 5)
player_to_pick, projected_roto_stats = zipsT.draft_from_list_and_find_best_pick(draft_pick_file = 'TestPicksround4.xlsx', path_list = path_drafts)

Team 1 picking Mike Trout for OF
Team 2 picking Mookie Betts for OF
Team 3 picking Alex Bregman for SS
Team 4 picking Anthony Rendon for 3B
Team 5 picking Nolan Arenado for 3B
Team 6 picking Jacob deGrom for SP
Team 7 picking Christian Yelich for OF
Team 8 picking Gerrit Cole for SP
Team 9 picking Cody Bellinger for OF
Team 10 picking Juan Soto for OF
Team 11 picking Justin Verlander for SP
Team 12 picking Francisco Lindor for SS
Team 12 picking Jose Altuve for 2B
Team 11 picking Max Scherzer for SP
Team 10 picking Ronald Acuna for OF
Team 9 picking Trevor Story for SS
Team 8 picking Charlie Blackmon for OF
Team 7 picking Whit Merrifield for 2B
Team 6 picking Freddie Freeman for 1B
Team 5 picking Trea Turner for SS
Team 4 picking Bryce Harper for OF
Team 3 picking Chris Sale for SP
Team 2 picking Walker Buehler for SP
Team 1 picking Shane Bieber for SP
Team 1 picking Stephen Strasburg for SP
Team 2 picking Josh Hader for RP
Team 3 picking Rafael Devers for 3B
Team 4 picking Fernando Ta

In [8]:
# Projected_roto_stats contains the estimated results from all teams given the recommented pick

projected_stats = projected_roto_stats[0]
projected_standings = projected_roto_stats[3]
projected_place = projected_roto_stats[4]
projected_scores = projected_roto_stats[5]

In [9]:
# The projected_stats give the raw numbers

projected_stats

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BB,SO,ERA,WHIP,BSV
0,707.0,730.0,241.0,22.0,236.0,776.0,105.0,504.0,0.273275,0.846712,67.0,41.0,4.0,4.0,52.0,270.0,1104.0,3.505703,1.136949,4.0
1,653.0,634.0,207.0,16.0,228.0,673.0,68.0,414.0,0.271854,0.846949,47.0,27.0,2.0,2.0,38.0,187.0,835.0,3.446524,1.105723,3.0
2,857.0,854.0,301.0,27.0,262.0,836.0,100.0,631.0,0.270531,0.834564,60.0,40.0,5.0,5.0,25.0,276.0,1007.0,3.540716,1.164106,2.0
3,648.0,583.0,225.0,19.0,242.0,759.0,43.0,474.0,0.262929,0.850685,41.0,31.0,1.0,1.0,90.0,218.0,715.0,3.607780,1.194868,7.0
4,719.0,804.0,263.0,37.0,204.0,719.0,115.0,449.0,0.276097,0.817787,42.0,28.0,1.0,1.0,46.0,197.0,681.0,3.833472,1.205383,3.0
5,676.0,789.0,259.0,30.0,162.0,622.0,126.0,421.0,0.272987,0.792400,48.0,30.0,2.0,2.0,0.0,222.0,855.0,3.310368,1.134865,0.0
6,624.0,705.0,236.0,25.0,167.0,619.0,83.0,451.0,0.273046,0.809749,44.0,30.0,2.0,2.0,27.0,232.0,823.0,3.479306,1.191097,2.0
7,665.0,765.0,232.0,33.0,175.0,628.0,83.0,435.0,0.274793,0.806534,64.0,34.0,3.0,3.0,30.0,225.0,1076.0,3.260024,1.072944,3.0
8,763.0,817.0,277.0,27.0,237.0,820.0,68.0,512.0,0.279630,0.844317,45.0,31.0,2.0,2.0,58.0,233.0,742.0,3.808917,1.249715,4.0
9,617.0,616.0,200.0,19.0,204.0,662.0,53.0,481.0,0.274444,0.855069,44.0,30.0,1.0,1.0,44.0,226.0,695.0,3.601425,1.231655,4.0


In [10]:
# And projected_scores show how they translate to roto scores

projected_scores

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BBP,SO,ERA,WHIP,BSV
0,9.0,7.0,7.0,4.0,9.0,10.0,10.0,10.0,7.0,9.0,12.0,1.0,10.5,10.5,10.0,2.0,12.0,7.0,8.0,3.5
1,5.0,4.0,3.0,1.0,8.0,7.0,3.5,2.0,4.0,10.0,7.0,12.0,6.5,6.5,6.0,12.0,7.0,9.0,10.0,7.0
2,12.0,12.0,12.0,6.5,12.0,12.0,8.5,12.0,3.0,7.0,9.0,2.0,12.0,12.0,2.0,1.0,9.0,6.0,7.0,10.0
3,4.0,1.0,4.0,2.5,11.0,9.0,1.0,8.0,1.0,11.0,1.0,6.5,2.5,2.5,12.0,9.0,4.0,4.0,5.0,1.0
4,10.0,10.0,10.0,12.0,6.5,8.0,11.0,6.0,10.0,5.0,2.5,11.0,2.5,2.5,8.0,11.0,2.0,2.0,4.0,7.0
5,7.0,9.0,9.0,8.0,2.0,4.0,12.0,3.0,5.0,2.0,8.0,9.0,6.5,6.5,1.0,8.0,8.0,11.0,9.0,12.0
6,3.0,5.0,6.0,5.0,3.0,3.0,5.5,7.0,6.0,4.0,4.5,9.0,6.5,6.5,3.0,5.0,6.0,8.0,6.0,10.0
7,6.0,8.0,5.0,11.0,4.0,5.0,5.5,5.0,9.0,3.0,11.0,4.0,9.0,9.0,5.0,7.0,10.0,12.0,12.0,7.0
8,11.0,11.0,11.0,6.5,10.0,11.0,3.5,11.0,12.0,8.0,6.0,6.5,6.5,6.5,11.0,4.0,5.0,3.0,2.0,3.5
9,2.0,2.0,1.0,2.5,6.5,6.0,2.0,9.0,8.0,12.0,4.5,9.0,2.5,2.5,7.0,6.0,3.0,5.0,3.0,3.5


In [11]:
# Finally, the projected_standings shows the resulting standings

projected_standings

2     167.0
0     158.5
8     149.0
7     147.5
4     141.0
5     140.0
1     130.5
10    112.5
6     112.0
11    105.0
3     100.0
9      97.0
dtype: float64

In [12]:
# But if you just want to know how your team finishes, that's projected_place

projected_place

5

### You can simulate drafts as well. Naive drafts simply picks the next eligible (i.e., open roster spot) player on the ranked list. 

In [13]:
# Try a quick draft: 4 teams, limited roster spots, naive (i.e., next available pick on ranked list) picks.

naive_draft = True
zipsS= Draft(player_rankings['ZiPS'],draft_position = 2,number_teams = 4, roster_spots = {'C':0,'1B':0,'2B':1, '3B':1,'SS':1,'OF':2,'UTIL':1,'SP':1,'RP':1,'P':1,'BN':1})
zipsS.draft_all(naive_draft = naive_draft, shuffle_picks = shuffle_picks)
print('Placement = '+str(zipsS.roto_placement))

Placement = 3


In [14]:
# Now try a naive draft again, but for full 12 team and all 24 roster spots.  Draft Position is 2 

naive_draft = True
zipsNaive_2 = Draft(player_rankings['ZiPS'], draft_position = 2)
zipsNaive_2.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zipsNaive_2.roto_placement))

Placement  = 3


In [15]:
# This time do the full-simulation, recommended picks version, i.e.; naive_draft = False
# Remember, it will try every remaining open position and pick the one whose resulting placement is highest.  
# Setting silent = False shows the decision logic for each pick (highest placement followed by highest score [placement, score])

naive_draft = False
zips_2 = Draft(player_rankings['ZiPS'], draft_position = 2)
zips_2.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks,silent=False)
print('Placement  = '+str(zips_2.roto_placement))

Team 1 Drafting Christian Yelich for OF
Stored Result for Pick 0 [2/2] Mookie Betts OF whose placement/score is [3, 132.5]
Stored Result for Pick 1 [2/3] Mike Trout UTIL whose placement/score is [3, 134.5]
Stored Result for Pick 2 [2/4] Ronald Acuna BN whose placement/score is [3, 135.5]
Stored Result for Pick 3 [2/6] Anthony Rendon 3B whose placement/score is [4, 130.5]
Stored Result for Pick 4 [2/7] Cody Bellinger 1B whose placement/score is [5, 100.0]
Stored Result for Pick 5 [2/9] Alex Bregman SS whose placement/score is [2, 161.5]
Stored Result for Pick 6 [2/11] Justin Verlander SP whose placement/score is [3, 150.5]
Stored Result for Pick 7 [2/13] Jacob deGrom P whose placement/score is [2, 166.0]
Stored Result for Pick 8 [2/15] Whit Merrifield 2B whose placement/score is [8, 154.5]
Not Storing Result for Pick 9 [2/27] Roberto Osuna RP
Not Storing Result for Pick 10 [2/185] J.T. Realmuto C
Team 2 picking Jacob deGrom for P
Team 3 Drafting Mookie Betts for OF
Team 4 Drafting Mike 

In [16]:
# Lets compare Naive True/False for the 7th pick

naive_draft = True
zipsNaive_7 = Draft(player_rankings['ZiPS'], draft_position = 7)
zipsNaive_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zipsNaive_7.roto_placement))

Placement  = 10


In [17]:
naive_draft = False
zips_7 = Draft(player_rankings['TheBat'], draft_position = 2)
zips_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zips_7.roto_placement))

Team 2 picking Gerrit Cole for SP
Team 2 picking J.T. Realmuto for C
Team 2 picking Charlie Blackmon for OF
Team 2 picking Ozzie Albies for 2B
Team 2 picking Austin Meadows for OF
Team 2 picking Carlos Correa for SS
Team 2 picking Noah Syndergaard for SP
Team 2 picking Aroldis Chapman for RP
Team 2 picking Corey Seager for UTIL
Team 2 picking Raisel Iglesias for OF
Team 2 picking Mike Moustakas for 3B
Team 2 picking Brandon Lowe for BN
Team 2 picking Zac Gallen for P
Team 2 picking Sean Doolittle for P
Team 2 picking Sean Manaea for P
Team 2 picking Yonny Chirinos for RP
Team 2 picking Joc Pederson for 1B
Placement  = 5


In [18]:
naive_draft = False
zips_7 = Draft(player_rankings['TheBat'], draft_position = 2)
zips_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zips_7.roto_placement))

Team 2 picking Gerrit Cole for SP
Team 2 picking J.T. Realmuto for C
Team 2 picking Charlie Blackmon for OF
Team 2 picking Ozzie Albies for 2B
Team 2 picking Austin Meadows for OF
Team 2 picking Carlos Correa for SS
Team 2 picking Noah Syndergaard for SP
Team 2 picking Aroldis Chapman for RP
Team 2 picking Corey Seager for UTIL
Team 2 picking Raisel Iglesias for OF
Team 2 picking Mike Moustakas for 3B
Team 2 picking Brandon Lowe for BN
Team 2 picking Zac Gallen for P
Team 2 picking Sean Doolittle for P
Team 2 picking Sean Manaea for P
Team 2 picking Yonny Chirinos for RP
Team 2 picking Joc Pederson for 1B
Placement  = 5


In [19]:
# And for the 11th pick

naive_draft = True
zipsNaive_11 = Draft(player_rankings['ZiPS'], draft_position = 11)
zipsNaive_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zipsNaive_11.roto_placement))

Placement  = 8


In [20]:
naive_draft = False
zips_11 = Draft(player_rankings['ZiPS'], draft_position = 11)
zips_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zips_11.roto_placement))

Team 11 picking Whit Merrifield for 2B
Team 11 picking Jacob deGrom for SP
Team 11 picking Austin Meadows for OF
Team 11 picking Keston Hiura for UTIL
Team 11 picking Nick Castellanos for OF
Team 11 picking Sean Doolittle for RP
Team 11 picking Adam Ottavino for RP
Team 11 picking Andrew Benintendi for OF
Team 11 picking Tommy Edman for 3B
Team 11 picking Luis Severino for SP
Team 11 picking Yu Darvish for P
Team 11 picking J.T. Realmuto for C
Team 11 picking Sergio Romo for P
Team 11 picking Dustin May for BN
Team 11 picking Eric Hosmer for SS
Team 11 picking Christian Walker for 1B
Team 11 picking Charlie Morton for P
Placement  = 7


In [21]:
# We can do simlated drafts for different projection systems.  Above was all with ZiPs.  Lets try a Steamer, with custom league settings.

number_teams = 6
naive_draft  = True
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':2}
steam_custom = Draft(player_rankings['ZiPS'], number_teams=number_teams, roster_spots=roster_spots)
steam_custom.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_custom.roto_placement))

Placement  = 2


In [22]:
# And compare Naive to the recommendations from the simulation

naive_draft = False
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':2}
steam_custom = Draft(player_rankings['ZiPS'], number_teams=number_teams, roster_spots=roster_spots)
steam_custom.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_custom.roto_placement))

Team 2 picking Mookie Betts for OF
Team 2 picking DJ LeMahieu for 1B
Team 2 picking Francisco Lindor for SS
Team 2 picking Jack Flaherty for SP
Swapping UTIL with OF
Team 2 picking Yordan Alvarez for OF
Team 2 picking Bryce Harper for OF
Team 2 picking Chris Sale for SP
Team 2 picking Yoan Moncada for 3B
Team 2 picking Keone Kela for RP
Team 2 picking Jorge Polanco for BN
Team 2 picking Javier Baez for UTIL
Team 2 picking Zack Greinke for P
Team 2 picking Eloy Jimenez for BN
Team 2 picking Adam Ottavino for P
Team 2 picking Liam Hendriks for RP
Team 2 picking Max Muncy for 2B
Team 2 picking Ian Kennedy for P
Team 2 picking Gary Sanchez for C
Placement  = 1


In [23]:
# And lets compare Naive True/False for different draft positions

naive_draft = True
steam_2= Draft(player_rankings['Steamer'], draft_position = 2)
steam_2.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_2.roto_placement))

Placement  = 12


In [24]:
naive_draft = False
steam_2= Draft(player_rankings['Steamer'], draft_position = 2)
steam_2.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_2.roto_placement))

Team 2 picking Gerrit Cole for SP
Team 2 picking Gleyber Torres for OF
Team 2 picking Starling Marte for OF
Team 2 picking Patrick Corbin for 3B
Team 2 picking Paul Goldschmidt for 1B
Team 2 picking Jose Abreu for OF
Team 2 picking Eugenio Suarez for UTIL
Team 2 picking Jeff McNeil for 2B
Team 2 picking Corey Seager for SS
Team 2 picking Jesus Luzardo for SP
Team 2 picking Ken Giles for RP
Team 2 picking Salvador Perez for C
Team 2 picking Lorenzo Cain for BN
Team 2 picking Joe Jimenez for RP
Team 2 picking Sean Doolittle for P
Team 2 picking Griffin Canning for P
Team 2 picking Joe Musgrove for P
Placement  = 6


In [25]:
naive_draft = True
steam_7= Draft(player_rankings['Steamer'], draft_position = 7)
steam_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_7.roto_placement))

Placement  = 10


In [26]:
naive_draft = False
steam_7= Draft(player_rankings['Steamer'], draft_position = 7)
steam_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_7.roto_placement))

Team 7 picking Freddie Freeman for 1B
Team 7 picking J.D. Martinez for OF
Team 7 picking Shane Bieber for SP
Team 7 picking J.T. Realmuto for C
Team 7 picking Yu Darvish for SP
Team 7 picking Bo Bichette for SS
Team 7 picking Joey Gallo for OF
Team 7 picking Jeff McNeil for 3B
Swapping UTIL with OF
Team 7 picking Michael Conforto for OF
Team 7 picking Eduardo Escobar for 2B
Team 7 picking Trey Mancini for UTIL
Team 7 picking Miguel Sano for BN
Team 7 picking Kenta Maeda for RP
Team 7 picking Lance McCullers Jr. for P
Team 7 picking Carlos Martinez for RP
Team 7 picking Will Smith for P
Team 7 picking Joe Musgrove for P
Placement  = 1


In [27]:
naive_draft = True
steam_11= Draft(player_rankings['Steamer'], draft_position = 11)
steam_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_11.roto_placement))

Placement  = 2


In [28]:
naive_draft = False
steam_11= Draft(player_rankings['Steamer'], draft_position = 11)
steam_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_11.roto_placement))

Team 11 picking Justin Verlander for OF
Team 11 picking Freddie Freeman for 1B
Team 11 picking Ozzie Albies for 2B
Team 11 picking J.T. Realmuto for C
Team 11 picking Mike Clevinger for 3B
Team 11 picking Matt Olson for BN
Team 11 picking Gary Sanchez for UTIL
Team 11 picking Carlos Correa for SS
Team 11 picking Liam Hendriks for RP
Team 11 picking Dinelson Lamet for SP
Team 11 picking Lorenzo Cain for SP
Team 11 picking Kyle Schwarber for OF
Team 11 picking Sean Manaea for P
Team 11 picking Andrew Heaney for P
Team 11 picking Mallex Smith for OF
Team 11 picking Ian Kennedy for RP
Team 11 picking Marcus Stroman for P
Placement  = 7


In [29]:
# And finally, 7th pick using TheBat projection system. 

naive_draft = True
thebat_7= Draft(player_rankings['TheBat'], draft_position = 7)
thebat_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(thebat_7.roto_placement))

Placement  = 2


In [30]:
naive_draft = False
thebat_7= Draft(player_rankings['TheBat'], draft_position = 7)
thebat_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(thebat_7.roto_placement))

Swapping UTIL with 3B
Team 7 picking Nolan Arenado for 3B
Team 7 picking Jose Ramirez for UTIL
Team 7 picking Shane Bieber for SP
Team 7 picking Chris Sale for SP
Team 7 picking Eugenio Suarez for BN
Team 7 picking Zack Greinke for P
Team 7 picking Marcell Ozuna for OF
Team 7 picking Carlos Santana for 1B
Team 7 picking Mike Soroka for P
Team 7 picking Willson Contreras for C
Team 7 picking Mike Moustakas for 2B
Team 7 picking Scott Kingery for OF
Team 7 picking Tim Anderson for SS
Team 7 picking Justin Upton for OF
Team 7 picking Mark Melancon for RP
Team 7 picking Jesus Luzardo for RP
Team 7 picking Mike Foltynewicz for P
Placement  = 1
